<img src="./img/header.png">
# Globus SDK
https://github.com/globusonline/globus-sdk-python


# Globus SDK Docs
http://globusonline.github.io/globus-sdk-python/


# Globus CLI
https://github.com/globusonline/globus-cli

### Some endpoints and associated endpoint IDs for the tutorial
go#ep1: ddb59aef-6d04-11e5-ba46-22000b92c6ec <br>
go#ep2: ddb59af0-6d04-11e5-ba46-22000b92c6ec


In [ ]:
from __future__ import print_function # for python 2
from globus_sdk import TransferClient

tutorial_endpoints = {"go#ep1":"ddb59aef-6d04-11e5-ba46-22000b92c6ec",
                       "go#ep2":"ddb59af0-6d04-11e5-ba46-22000b92c6ec"}

# Transfer Client

## Instantiate Transfer Client

In [ ]:
tc = TransferClient() # uses transfer_token from the config file

## Low-level Interface

In [ ]:
print("==== My Endpoints ====")
r = tc.get("/endpoint_search?filter_scope=my-endpoints")
for epdict in r.data["DATA"]:
    print("Display Name:%s\nID:%s\n"%(epdict["display_name"], epdict["id"]))

## High-level Interface
provides iterators for list responses

In [ ]:
print("==== My Endpoints ====")
for r in tc.endpoint_search(filter_scope='my-endpoints'):
    print("Display Name:%s\nID:%s\n"%(r.data["display_name"], r.data["id"]))

## Endpoint search by name

In [ ]:
search_str = "go#ep2"
print("==== Displaying endpoint matches for search: '%s' ===="%search_str)
for r in tc.endpoint_search(search_str):
    print("Display Name:%s\nID:%s\n"%(r.data["display_name"], r.data["id"]))

# File operations

### Path encoding & UTF gotchas

### Autoactivate Endpoints
Here we activate the tutorial endpoints go#ep1 and go#ep2 using their endpoint_ids

In [ ]:
#help(tc.endpoint_activate)
r_go1 = tc.endpoint_autoactivate(tutorial_endpoints['go#ep1'])
r_go2 = tc.endpoint_autoactivate(tutorial_endpoints['go#ep2'])

### Get a directory listing

In [ ]:
# help(tc.operation_ls)
endpoint_id = tutorial_endpoints['go#ep1'] 
endpoint_path = '/~/shared_dir/'
r = tc.operation_ls(endpoint_id,path=endpoint_path)
print("==== Endpoint_ls for endpoint %s %s ===="%(endpoint_id,endpoint_path))
for item in r.data['DATA']:
    print("%s: %s [%s]"%(item['type'],item['name'],item['size']))

### Make directory

In [ ]:
#help(tc.operation_mkdir)
endpoint_id = tutorial_endpoints['go#ep1'] 
endpoint_path = '/~/shared_dir/new_dir'
r = tc.operation_mkdir(endpoint_id,path=endpoint_path)

### Rename

In [ ]:
help(tc.operation_rename)

# Task submission

### Transfer w/ and w/o sync
Doesn't work yet...

In [ ]:
help(tc.submit_transfer)
source_endpoint = tutorial_endpoints['go#ep1']
source_path = '/~/shared_dir/'

dest_endpoint = tutorial_endpoints['go#ep2']
dest_path = '/~/'

label = "My tutorial transfer"

transfer_items = tc.make_submit_transfer_item()

#tc.make_submit_transfer_data(source_endpoint, dest_endpoint, label)

### Delete

# Task management

### Get task_list

### Get task by id

In [ ]:
help(tc.get_task)

### Update task by id (label, deadline)

In [ ]:
help(tc.update_task)

### Cancel task by id

In [ ]:
help(tc.cancel_task)

### Get event list for task

### Get task pause info

# Bookmarks

# Create a Bookmark

In [ ]:
bookmark_name = "My Tutorial Bookmark"
endpoint_id = tutorial_endpoints['go#ep1']
endpoint_path = '/'
r = tc.create_bookmark({"endpoint_id":endpoint_id, "path":endpoint_path,"name":bookmark_name})
r

# Delete a Bookmark

In [ ]:
tc.delete_bookmark(r.data['id'])

### CRUD + list

# Shared endpoints

### Access_manager role

### Get list off access rules

### Get access rule by id

### Create access rule

### Update access rule

### Delete access rule